In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Resizing, Rescaling

In [ ]:
from tensorflow.keras.applications import MobileNetV3Small

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

## Exploratory Visualization and Analysis

In [ ]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
x_train.shape

In [ ]:
single_image = x_train[0]

In [ ]:
single_image.shape

In [ ]:
plt.imshow(single_image);

## Labeling

In [ ]:
y_train

In [ ]:
y_train.shape

## Scaling

In [ ]:
x_train.max()

In [ ]:
x_train.min()

## Creating a Prefab Model and Training

In [ ]:
base_model = MobileNetV3Small(include_top=False, input_shape=(32,32,3), weights='imagenet')
base_model = Model(inputs=base_model.input, outputs=Flatten()(base_model.output))

model = Sequential()

model.add(Resizing(32,32))
model.add(Rescaling(1.0/255))

model.add(base_model)

model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=10, activation='softmax'))

# Train just the last 10 layers
# for layer in model.layers:
#     layer.trainable = False

# for layer in model.layers[-10:]:
#     layer.trainable = True

# Block the base_model layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

In [ ]:
model.fit(x=x_train, y=y_train, epochs=20, batch_size=16, validation_data=(x_test,y_test), callbacks=[early_stop])

## Evaluation

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['accuracy','val_accuracy']].plot();

In [ ]:
losses[['loss','val_loss']].plot();

In [ ]:
print(model.metrics_names)
print(model.evaluate(x_test, y_test, verbose=0))

In [ ]:
predictions = np.argmax(model.predict(x_test), axis=1)
predictions

In [ ]:
y_test

In [ ]:
print(classification_report(y_test, predictions))

## Predict Classes

In [ ]:
my_img = x_test[100]
plt.imshow(my_img.reshape(32,32,3));

In [ ]:
prediction = np.argmax(model.predict(my_img.reshape(1,32,32,3)), axis=1)
prediction